In [1]:
import os
os.getcwd()

'/home/gaowei/.cache/bazel/_bazel_gaowei/468ff0ee32e8cf69d70c757ab23dfef8/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/ipython_spot/parse_data.runfiles/com_nvidia_isaac/ipython_spot'

In [2]:
#go back to library path.
os.chdir("..")
os.getcwd()

'/home/gaowei/.cache/bazel/_bazel_gaowei/468ff0ee32e8cf69d70c757ab23dfef8/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/ipython_spot/parse_data.runfiles/com_nvidia_isaac'

In [11]:
CASK_PATH='/home/gaowei/tmp_data/demo000'
EXPORT_DIR="/home/gaowei/tmp_data"
from engine.pyalice import Cask, Message
import io
from PIL import Image
cask = Cask(CASK_PATH)

# List all channels recorded
for channel in cask.channels:
    print (channel)
    if channel.name == "mid_color":
        for idx, msg in enumerate(channel):                        # Goes through every messages one by one in recorded order
           print(msg.proto)
           print(msg.tensor)
#            Image.fromarray(msg.tensor).save(io.BytesIO(), 'png')
           Image.fromarray(msg.tensor).save(os.path.join(EXPORT_DIR, f"{idx}.png"))
           print(msg.uuid)
           print(msg.acqtime)
           print(msg.pubtime)

{name: left_color, #: 2454, tensor: yes}
{name: left_depth, #: 2454, tensor: yes}
{name: right_depth, #: 2468, tensor: yes}
{name: mid_color, #: 2443, tensor: yes}
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[131 121 128]
  [132 122 129]
  [147 130 127]
  ...
  [209 129  78]
  [207 128  75]
  [207 128  75]]

 [[133 117 125]
  [142 126 134]
  [149 132 128]
  ...
  [206 127  74]
  [206 127  72]
  [207 128  73]]

 [[133 117 121]
  [140 124 128]
  [155 132 128]
  ...
  [209 127  73]
  [207 124  70]
  [206 123  69]]

 ...

 [[ 72  84  82]
  [ 61  73  72]
  [ 56  78  74]
  ...
  [103 120 127]
  [102 121 127]
  [ 95 114 120]]

 [[ 70  85  82]
  [ 61  76  73]
  [ 61  83  76]
  ...
  [107 117 127]
  [ 99 115 124]
  [ 98 114 123]]

 [[

280a24c4-1060-11eb-ab5c-d723bbd3724f
494264630380
494121832576
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[138 109 125]
  [140 111 128]
  [149 126 124]
  ...
  [207 132  83]
  [204 132  73]
  [203 130  72]]

 [[141 106 121]
  [144 110 125]
  [149 129 125]
  ...
  [207 130  82]
  [206 129  77]
  [205 128  76]]

 [[141 107 119]
  [148 114 126]
  [152 128 132]
  ...
  [209 128  80]
  [211 128  81]
  [209 126  79]]

 ...

 [[ 60  79  87]
  [ 60  79  87]
  [ 62  73  81]
  ...
  [104 118 122]
  [104 118 122]
  [101 115 120]]

 [[ 61  77  89]
  [ 59  75  87]
  [ 66  73  83]
  ...
  [109 119 127]
  [103 113 124]
  [100 110 120]]

 [[ 63  77  83]
  [ 64  78  84]
  [ 69  76  85]
  ...
  [108 116 123]
  [105 111 124]
  [101 108 120]]]


286814ee-1060-11eb-ab5c-d723bbd3724f
494865630316
494737466013
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[136  96 120]
  [148 109 133]
  [156 124 120]
  ...
  [210 130  79]
  [209 124  87]
  [210 125  88]]

 [[137  99 116]
  [157 119 135]
  [156 127 122]
  ...
  [207 131  76]
  [209 126  79]
  [209 126  79]]

 [[146 109 121]
  [157 119 131]
  [162 130 126]
  ...
  [206 128  70]
  [207 122  65]
  [209 125  68]]

 ...

 [[ 51  76  81]
  [ 52  78  82]
  [ 54  81  79]
  ...
  [105 116 124]
  [103 114 120]
  [104 115 121]]

 [[ 54  76  84]
  [ 54  76  84]
  [ 61  84  83]
  ...
  [103 116 125]
  [102 113 114]
  [103 115 115]]

 [[ 58  84  84]
  [ 54  81  81]
  [ 62  83  83]
  ...
  [ 99 115 124]
  [101 111 119]
  [100 110 118]]]


28c19726-1060-11eb-ab5c-d723bbd3724f
495465630316
495324070247
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[135  97 132]
  [146 108 142]
  [152 127 138]
  ...
  [216 128  72]
  [216 125  71]
  [210 119  65]]

 [[138 105 128]
  [150 117 140]
  [153 125 137]
  ...
  [210 129  70]
  [213 128  73]
  [207 122  67]]

 [[143 111 127]
  [152 120 136]
  [157 126 137]
  ...
  [209 130  74]
  [212 130  76]
  [209 127  73]]

 ...

 [[ 59  76  83]
  [ 59  76  83]
  [ 56  79  83]
  ...
  [ 98 115 117]
  [ 94 118 119]
  [ 97 120 122]]

 [[ 60  77  82]
  [ 58  75  80]
  [ 55  78  80]
  ...
  [100 117 120]
  [ 97 117 118]
  [ 98 119 120]]

 [[ 63  80  80]
  [ 59  77  77]
  [ 53  80  78]
  ...
  [102 122 123]
  [100 117 120]
  [ 99 116 118]]]


291e782e-1060-11eb-ab5c-d723bbd3724f
496065630316
495932763014
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[137 116 118]
  [140 120 121]
  [147 131 123]
  ...
  [215 127  67]
  [209 131  68]
  [206 128  66]]

 [[136 112 115]
  [143 119 122]
  [152 130 126]
  ...
  [210 129  66]
  [209 128  65]
  [208 127  64]]

 [[146 103 113]
  [159 116 126]
  [159 133 128]
  ...
  [211 123  68]
  [208 126  68]
  [207 125  66]]

 ...

 [[ 61  80  88]
  [ 56  76  84]
  [ 55  74  82]
  ...
  [ 99 116 118]
  [101 118 123]
  [101 118 123]]

 [[ 63  79  88]
  [ 60  76  86]
  [ 61  74  85]
  ...
  [104 117 124]
  [101 118 121]
  [101 118 121]]

 [[ 65  73  91]
  [ 65  73  91]
  [ 66  76  86]
  ...
  [101 111 119]
  [102 112 120]
  [101 111 119]]]


...

4f63b922-1060-11eb-ab5c-d723bbd3724f
560286630252
560140044126
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[209 189 199]
  [210 191 200]
  [213 190 201]
  ...
  [190 178 182]
  [193 178 181]
  [195 180 183]]

 [[208 192 202]
  [209 193 203]
  [209 193 201]
  ...
  [187 181 184]
  [190 181 183]
  [192 183 185]]

 [[206 197 201]
  [205 195 200]
  [209 196 203]
  ...
  [187 184 183]
  [188 185 184]
  [189 186 185]]

 ...

 [[122 147 154]
  [119 145 152]
  [110 142 148]
  ...
  [ 65 120 124]
  [ 64 119 123]
  [ 60 116 119]]

 [[124 149 159]
  [122 147 156]
  [119 144 154]
  ...
  [ 64 119 123]
  [ 61 119 123]
  [ 58 116 120]]

 [[124 149 161]
  [122 146 158]
  [122 147 156]
  ...
  [ 66 117 125]
  [ 63 118 123]
  [ 59 114 120]]]


4fbfacfa-1060-11eb-ab5c-d723bbd3724f
560886630252
560742664666
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[208 187 191]
  [208 187 191]
  [205 186 196]
  ...
  [191 173 178]
  [192 174 179]
  [194 175 180]]

 [[209 188 192]
  [208 187 191]
  [205 189 197]
  ...
  [188 176 180]
  [188 176 178]
  [189 177 179]]

 [[212 187 197]
  [210 184 195]
  [210 191 198]
  ...
  [188 176 180]
  [188 176 178]
  [189 177 179]]

 ...

 [[118 139 153]
  [116 138 151]
  [121 134 145]
  ...
  [ 69 111 123]
  [ 70 108 122]
  [ 71 110 123]]

 [[122 137 152]
  [122 137 152]
  [121 136 149]
  ...
  [ 71 110 123]
  [ 70 106 119]
  [ 72 108 121]]

 [[125 137 151]
  [124 136 150]
  [118 137 147]
  ...
  [ 71 110 123]
  [ 72 111 123]
  [ 73 112 124]]]


501b6f18-1060-11eb-ab5c-d723bbd3724f
561487630188
561344011859
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[212 180 196]
  [209 177 193]
  [202 186 194]
  ...
  [189 176 183]
  [189 176 185]
  [189 176 185]]

 [[208 183 194]
  [207 182 192]
  [204 188 194]
  ...
  [186 177 180]
  [186 177 180]
  [188 178 181]]

 [[211 186 196]
  [211 186 196]
  [205 186 194]
  ...
  [186 177 180]
  [185 176 178]
  [185 176 178]]

 ...

 [[119 141 150]
  [118 140 149]
  [118 144 151]
  ...
  [ 74 116 126]
  [ 67 116 120]
  [ 61 111 115]]

 [[119 141 150]
  [118 140 149]
  [117 142 150]
  ...
  [ 71 114 124]
  [ 64 116 121]
  [ 60 112 117]]

 [[121 143 154]
  [122 144 155]
  [119 147 155]
  ...
  [ 67 116 125]
  [ 62 114 119]
  [ 61 113 118]]]


50771a52-1060-11eb-ab5c-d723bbd3724f
562089630316
561944771769
( image = (
    elementType = uint8,
    rows = 360,
    cols = 640,
    channels = 3,
    dataBufferIndex = 0 ),
  colorSpace = rgb,
  pinhole = (
    rows = 360,
    cols = 640,
    focal = (x = 460.13446044921875, y = 459.76773071289062),
    center = (x = 184.73890686035156, y = 323.8455810546875) ) )
[[[205 194 208]
  [205 194 208]
  [214 191 205]
  ...
  [175 169 172]
  [182 164 166]
  [182 164 166]]

 [[208 194 207]
  [208 194 207]
  [213 194 206]
  ...
  [175 169 174]
  [179 166 170]
  [179 166 170]]

 [[212 195 206]
  [212 195 206]
  [212 195 206]
  ...
  [175 169 172]
  [181 165 173]
  [181 165 173]]

 ...

 [[109 131 142]
  [109 131 142]
  [103 135 141]
  ...
  [ 23  64  66]
  [ 25  60  62]
  [ 24  59  61]]

 [[113 128 142]
  [114 129 144]
  [108 133 140]
  ...
  [ 24  65  69]
  [ 25  62  66]
  [ 23  61  64]]

 [[116 131 146]
  [116 131 146]
  [112 133 145]
  ...
  [ 23  73  74]
  [ 24  67  70]
  [ 24  67  70]]]


KeyboardInterrupt: 